# Bitácora de exploración
## Collective Intelligence - Making Recommendations

### Filtros colaborativos

Se desarrollará un sistema de recomendación basado coincidencias de un usuario con otros.

Un algorítmo de *filtrado colaborativo* usualmente funciona buscando en un grupo grande de personas y encontrando un conjunto mas pequeño con gustos similares a los de cierto usuario.

En general, filtrado colaborativo es el proceso de filtrado de información o patrones usando técnicas que involucran la colaboración entre múltiples agentes, puntos de vista, fuentes de datos, etc.

### Recolectando preferencias

Lo primero que se tiene que considerar es una representación que nos permita hablar de **personas** y sus **gustos** sobre determinados **ítems**. En este caso las personas serán críticos de películas, los gustos se representarán como las calificaciones que le asignan a películas, las cuales serán los ítems.

La representación con la que se trabajará será la de diccionarios anidados, si `d[a,b]` es un diccionario que relaciona `a` con `b`, la estructura que utilizaremos tiene la forma:

```
d[Nombre de persona, d[Nombre de película, Calificación]
```

Tanto el nombre del crítico como el nombre de la película son cadenas de caracteres; la calificación se representa como un valor numérico entre 1 y 5

In [49]:
# Un diccionario de críticos de cine con sus calificaciones de un
# conjunto pequeño de películas
critics = {
    "Lisa Rose" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 3.5,
        "You, Me and Dupree" : 2.5,
        "The Night Listener" : 3.0
    },
    "Gene Seymour" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 1.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 3.5
    },
    "Michael Phillips" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.0,
        "Superman Returns"   : 3.5,
        "The Night Listener" : 4.0
    },
    "Claudia Puig" : {
        "You, Me and Dupree" : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 4.0,
        "The Night Listener" : 4.5
    },
    "Mick LaSalle" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "Just My Luck"       : 2.0,
        "Superman Returns"   : 3.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 2.0
    },
    "Jack Matthews" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "You, Me and Dupree" : 3.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0
    },
    "Toby" : {
        "Snakes on a Plane"  : 4.5,
        "You, Me and Dupree" : 1.0,
        "Superman Returns"   : 4.0
    }
}

### Encontrando usuarios similares

Para determinar que tan similar son los gustos de dos personas se pueden utilizar varias métricas, las cuales llamamos *puntuaciones de similitud*

En este ejercicio se implementarán dos: la *distancia Euclideana* y la *correlación de Pearson*

#### Puntuación de la distancia Euclideana

Considerando $n$ películas calificadas por una cantidad de críticos, podemos encontrar que tan similar son dos personas si tomamos un espacio $n$-dimensional en donde un punto es un crítico y sus coordenadas se determinan por la calificación que le asignó dicho crítico a las películas.

La distancia Euclideana en este caso será calculada entre dos críticos considerando las películas que calificaron en común.

$$f(c^{(a)}, c^{(b)}) = \sqrt{(c^{(a)}_1 - c^{(b)}_1)^2 + \dots (c^{(a)}_n - c^{(b)}_n)^2}$$

donde $c^{(r)}_{i}$ es la calificación que le asignó el crítico $r$ a la película $i$

In [50]:
# Obtiene una lista con los ítems en común entre dos usuarios
def shared_items(data, user_a, user_b):
    return [item for item in data[user_a] if item in data[user_b]]

In [51]:
shared_items(critics, "Gene Seymour", "Lisa Rose")

['Lady in the Water',
 'Snakes on a Plane',
 'Just My Luck',
 'Superman Returns',
 'You, Me and Dupree',
 'The Night Listener']

In [52]:
from math import sqrt

In [53]:
# Calcula la distancia euclideana entre dos usuarios en la base de datos
def euclidean_distance(data, user_a, user_b):
    return sqrt(sum([pow(data[user_a][item] - data[user_b][item], 2)
                     for item in shared_items(data, user_a, user_b)]))

In [54]:
euclidean_distance(critics, "Gene Seymour", "Lisa Rose")

2.3979157616563596

In [55]:
# Calcula similaridad = 1/(1+distancia)
def euclidean_similarity(data, user_a, user_b):
    return 1/(1+euclidean_distance(data, user_a, user_b))

In [56]:
euclidean_similarity(critics, "Gene Seymour", "Lisa Rose")

0.29429805508554946

#### Puntuación de la correlación de Pearson

Una manera mas sofisticada para determinar la similaridad entre los intereses de dos personas es usando el coeficiente de correlación de Pearson.

Considerando a dos críticos $a$ y $b$ podemos encontrar que tan similares son sus gustos si tomamos las películas que calificaron en común y las colocamos en un espacio bidimensional, en donde cada punto es $(c^{(a)}_i, c^{(b)}_i)$. Donde $c^{(r)}_i$ es la calificación que le asignó el crítico $r$ a la película $i$.

Se hace una regresión lineal con los puntos para obtener una recta con la que podemos determinar que tan cerca o lejos están las calificaciones de los críticos de ella.

In [57]:
def critics_ratings(data, user_a, user_b):
    shared = shared_items(data, user_a, user_b)
    c_a = [data[user_a][item] for item in shared]
    c_b = [data[user_b][item] for item in shared]
    n = len(shared)
    return c_a, c_b, n

In [58]:
c_a, c_b, n = critics_ratings(critics, "Gene Seymour", "Lisa Rose")
print "c_a = ", c_a
print "c_b = ", c_b
print "n   = ", n

c_a =  [3.0, 3.5, 1.5, 5.0, 3.5, 3.0]
c_b =  [2.5, 3.5, 3.0, 3.5, 2.5, 3.0]
n   =  6


Hay dos casos extremos que debemos considerar, primero el caso en el que dos críticos no tienen películas calificadas en común y luego el caso bizarro en el que la similaridad se indetermine (considere el caso de la similaridad de Pearson con "Luque" y "Moises".

En ambos casos se regresa 0 (lo que significa que no hay correlación entre estos dos críticos). Para el caso en el que se indetermina el resultado es por una división por cero, la cual puede ocurrir si se permiten las calificaciones de 0 o si los dos críticos tienen solo una película en común y uno de ellos la calificó con 1, es un caso peculiar que no sé como arreglar, así que lo dejaremos en que no hay correlación en este caso.

In [59]:
def pearson_similarity(data, user_a, user_b):
    c_a, c_b, n = critics_ratings(data, user_a, user_b)
    if n == 0: return 0
    sum_c_a = sum(c_a)
    sum_c_b = sum(c_b)
    sum_c_a_sq = sum(map(lambda x,y:x*y, c_a, c_a))
    sum_c_b_sq = sum(map(lambda x,y:x*y, c_b, c_b))
    sum_c_prod = sum(map(lambda x,y:x*y, c_a, c_b))
    numerator = sum_c_prod - (sum_c_a*sum_c_b/float(n))
    denominator = sqrt((sum_c_a_sq-pow(sum_c_a,2)/float(n))*
                       (sum_c_b_sq-pow(sum_c_b,2)/float(n)))
    if denominator == 0: return 0
    return numerator/denominator

In [61]:
pearson_similarity(critics, "Toby", "Lisa Rose")

0.9912407071619299

#### Ordenando usuarios

Se implementa una función que a partir de un usuario, regresa una lista ordenada de otros usuarios (de mas similares a menos similares).

In [72]:
def recommend_critics(data, user, similarity=pearson_similarity):
    return sorted([(similarity(data, user, other), other)
                  for other in data if other != user])[::-1]

In [79]:
recommend_critics(critics, "Toby")

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig'),
 (0.66284898035987, 'Jack Matthews'),
 (0.38124642583151164, 'Gene Seymour'),
 (-1.0, 'Michael Phillips')]

In [84]:
def top_recommended_critics(data, user, n=5, similarity=pearson_similarity):
    return recommend_critics(data, user, similarity)[:n]

In [85]:
top_recommended_critics(critics, "Toby", n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

### Recomendando ítems

Hasta ahora se ha mostrado como podemos determinar que crítico de película es el mas similar a un determinado usuario. Sin embargo, lo que se desea no es una recomendación de críticos, si no una recomendación de películas.

Para no depender únicamente del crítico mas similar al usuario, debemos de tomar en cuenta a varios críticos con varios grados de similaridad. Podemos ponderar linealmente las calificaciones de las películas con el grado de similaridad del crítico. De esta manera tomamos en cuenta a todos los críticos pero a los mas similares los tomamos más en cuenta que los menos similares.

In [80]:
def recommend_movies(data, user, similarity=pearson_similarity):
    totals = {}
    similarity_sums = {}
    for critic in data:
        if critic == user: continue
        critic_score = similarity(data, user, critic)
        if critic_score <= 0: continue
        for movie in data[critic]:
            if movie not in data[user] or data[user][movie] == 0:
                totals.setdefault(movie,0)
                totals[movie]+=data[critic][movie]*critic_score
                similarity_sums.setdefault(movie, 0)
                similarity_sums[movie] += critic_score
    rankings = [(total/similarity_sums[movie],movie)
               for movie, total in totals.items()]
    return sorted(rankings)[::-1]

In [81]:
recommend_movies(critics, "Toby")

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [82]:
recommend_movies(critics, "Toby", similarity=euclidean_similarity)

[(3.457128694491423, 'The Night Listener'),
 (2.778584003814924, 'Lady in the Water'),
 (2.4224820423619167, 'Just My Luck')]

### Coincidiendo productos

Otro mecanismo para recomendarle a un usuario películas es asociar a las películas entre sí. Similar a sugerencias parecidas a "A los críticos que les agradó esta película, también les agradó...".

In [86]:
def invert_data(data):
    result = {}
    for person in data:
        for item in data[person]:
            result.setdefault(item, {})
            result[item][person] = data[person][item]
    return result

In [91]:
movies = invert_data(critics)
movies

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.0,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [92]:
def recommend_similar_movies(data, movie, similarity=pearson_similarity):
    return sorted([(similarity(data, movie, other), other)
                  for other in data if other != movie])[::-1]

def top_recommended_similar_movies(data, movie, n=5, similarity=pearson_similarity):
    return recommend_similar_movies(data, movie, similarity)[:n]

In [93]:
top_recommended_similar_movies(movies, "Superman Returns")

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

### Filtrado basado en ítems

### Usando el conjunto de datos de MovieLens

In [94]:
import csv
import os.path

def read_dataset():
    print "Checking existance of MovieLens dataset..."
    if not( os.path.exists("MovieLens/movies.csv") or
            os.path.exists("MovieLens/ratings.csv")):
        print "  Couldn't find files, you need to put the movies.csv and ratings.csv files inside a MovieLens directory relative to this file\n"
        print "  Download link for MovieLens datasets is http://grouplens.org/datasets/movielens/"
        return None

    print "MovieLens data set found!"
    print "The reading of the dataset will take a few minutes..."
    movies_path = "MovieLens/movies.csv"
    movies = dict()
    with open(movies_path, "r") as file:
        reader = csv.DictReader(file)
        for entry in reader:
            movie_id = entry["movieId"]
            title = entry["title"]
            movies[movie_id] = title

    ratings_path = "MovieLens/ratings.csv"
    dataset = dict()
    with open(ratings_path, "r") as file:
        reader = csv.DictReader(file)
        for entry in reader:
            user_id = int(entry["userId"])
            movie_id = entry["movieId"]
            movie_title = movies[movie_id]
            rating = float(entry["rating"])
            if not dataset.has_key(user_id):
                dataset[user_id] = dict()
            dataset[user_id][movie_title] = rating
    return dataset

In [95]:
data = read_dataset()

Checking existance of MovieLens dataset...
MovieLens data set found!
The reading of the dataset will take a few minutes...


In [100]:
recommend_movies(data, 1)

[(5.000000000000001, 'Return to Source: The Philosophy of The Matrix (2004)'),
 (5.000000000000001, 'Requiem For The Big East (2014)'),
 (5.000000000000001, 'Hellhounds on My Trail (1999)'),
 (5.000000000000001, 'Fiancee, The (Parineeta) (1953)'),
 (5.000000000000001,
  'Consuming Kids: The Commercialization of Childhood (2008)'),
 (5.000000000000001, 'Babai (2015)'),
 (5.000000000000001, 'Ape (2012)'),
 (5.0, 'Zapatlela (1993)'),
 (5.0, 'When I Walk (2013)'),
 (5.0, "Weekend It Lives, The (Ax 'Em) (1992)"),
 (5.0, 'Wasted Youth (2011)'),
 (5.0, 'Warriors Two (1978)'),
 (5.0, 'Vertical Features Remake (1978)'),
 (5.0, 'Variety (1983)'),
 (5.0, "Tyler Perry's Temptation: Confessions of a Marriage Counselor (2013)"),
 (5.0, 'Tony 10 (2012)'),
 (5.0, 'Time Zero: The Last Year of Polaroid Film (2012)'),
 (5.0, 'The Village Had No Walls (1995)'),
 (5.0, 'The Rise and Rise of Bitcoin (2014)'),
 (5.0, 'The Professional: Golgo 13 (1983)'),
 (5.0, 'The Monkey King (1964)'),
 (5.0, 'The Misfit B